# 43 Sistema solar - final

As texturas do sistema solar foram obtidas do site: https://www.solarsystemscope.com/textures/

Modelo do ovni obtido no link: https://www.cgtrader.com/free-3d-models/vehicle/sci-fi/saucer-2-and-saucer-2-advanced

Modelo do meteoro obtido no link: https://www.cgtrader.com/free-3d-models/science/other/stone-meteor

Textura procedural do sol baseada no código presente no link: https://www.shadertoy.com/view/XljfRc

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [14]:
import glm
import time
import math
import numpy as np
import OpenGL.GL as gl
from PyQt5 import QtOpenGL, QtCore
from PyQt5.QtCore import QCoreApplication
from PyQt5.QtWidgets import QApplication

from cg.shader_programs.SimpleShaderProgram_v3 import SimpleShaderProgram
from cg.shader_programs.PhongShadingShaderProgram_v2 import PhongShadingShaderProgram
from cg.shader_programs.SunEffectShaderProgram import SunEffectShaderProgram
from cg.models.SphereMesh_v3 import SphereMesh
from cg.models.OrbitPolygon import OrbitPolygon
from cg.models.SquareMesh_v3 import SquareMesh
from cg.models.Torus2DMesh_v3 import Torus2DMesh
from cg.models.ConeMesh import ConeMesh
from cg.models.ObjLoader_v2 import ObjLoader
from cg.utils.Texture import Texture
from cg.renderers.ModelRenderer_v4 import ModelRenderer

AU = 1.496E8
planet_scale_factor = 1000.0
moon_scale_factor = planet_scale_factor * 0.05
sun_scale_factor = 50
skybox_size = 150.0

# classe para armazenar as informações do Sol
class SunInfo:
    def __init__(self, color, diameter):
        
        self.color = color
        self.diameter = (diameter / AU) * sun_scale_factor
        self.radius = self.diameter / 2.0

# classe para armazenar as informações dos planetas
class PlanetInfo:
    def __init__(self, color, diameter, orbitalPeriod, orbitalInclination, orbitSemiMajorAxis, orbitSemiMinorAxis, orbitFocus):
        
        earth_days = 365.2
        
        self.color = color
        self.diameter = (diameter / AU) * planet_scale_factor
        self.radius = self.diameter / 2.0
        self.velocity = earth_days / orbitalPeriod
        
        self.orbitalPeriod = orbitalPeriod
        self.orbitalInclination = orbitalInclination 
        self.orbitSemiMajorAxis = orbitSemiMajorAxis
        self.orbitSemiMinorAxis = orbitSemiMinorAxis
        self.orbitFocus = orbitFocus

# cria um dicionário contendo as informações dos planetas       
PlanetarySheet = {
       'sun': SunInfo(np.array([1.00, 1.00, 0.00, 1.0]), 1391900),
       'mercury': PlanetInfo(np.array([0.96, 0.90, 0.71, 1.0]), 4866, 88.0, 6.34, 0.3870, 0.3788, 0.0796),
       'venus': PlanetInfo(np.array([0.95, 0.82, 0.38, 1.0]), 12106, 224.7, 2.19, 0.7219, 0.7219, 0.0049),
       'earth': PlanetInfo(np.array([0.61, 0.79, 0.37, 1.0]), 12742, 365.2, 1.57, 1.0027, 1.0025, 0.0167),
       'moon': PlanetInfo(np.array([0.50, 0.50, 0.50, 1.0]), 3475, 27.3, 5.1, 0.0025718 * moon_scale_factor, 0.0021479 * moon_scale_factor, 0.00014537),
       'mars': PlanetInfo(np.array([0.88, 0.81, 0.61, 1.0]), 6760, 687.0, 1.67, 1.5241, 1.5173, 0.1424),
       'jupiter': PlanetInfo(np.array([0.83, 0.67, 0.53, 1.0]), 142984, 4331, 0.32, 5.2073, 5.2010, 0.2520),
       'saturn': PlanetInfo(np.array([0.89, 0.87, 0.63, 1.0]), 116438, 10747, 0.93, 9.5590, 9.5231, 0.5181),
       'uranus': PlanetInfo(np.array([0.00, 0.87, 0.95, 1.0]), 46940, 30589, 1.02, 19.1848, 19.1645, 0.9055),
       'neptune': PlanetInfo(np.array([0.00, 0.51, 0.89, 1.0]), 45432, 59800, 0.72, 30.0806, 30.0788, 0.2587),
       'pluto': PlanetInfo(np.array([0.62, 0.63, 0.64, 1.0]), 2274, 90560, 15.55, 39.5, 34.031, 9.8276)
       }

# classe para armazenar as informações gráficas em um único objeto
class GraphicObject:
    def __init__(self):
        self.model = None
        self.renderer = None
        self.modelMatrix = glm.mat4()
        self.color = np.array([1.0, 1.0, 1.0, 1.0])
        self.texture = None
        
class MyWidget(QtOpenGL.QGLWidget):

    def initializeGL(self):
        
        self.planetName = ['mercury', 'venus', 'earth', 'moon', 'mars', 'jupiter', 'saturn', 'uranus', 'neptune', 'pluto']
        self.orbits = {}
        self.planets = {}
        
        self.yaw   = -90.0
        self.pitch = -45.0
        
        # cria a matriz de visão
        self.cameraPos   = glm.vec3(0.0, 5.0, 5.0)
        self.cameraFront = glm.vec3(0.0, -1.0, -1.0)
        self.cameraUp    = glm.vec3(0.0, 1.0,  0.0)
        self.viewMatrix = glm.lookAt(self.cameraPos, self.cameraFront, self.cameraUp)
        
        # define uma transformação global
        self.globalTransform = glm.rotate(glm.mat4(), glm.radians(-90), glm.vec3(1.0, 0.0, 0.0))
        
        # cria os shader programs necessários   
        self.simpleShaderProgram = SimpleShaderProgram()
        self.phongShadingProgram = PhongShadingShaderProgram()
        self.sunEffectShaderProgram = SunEffectShaderProgram()
        
        # ativa o shader programa para permitir configurar uma cor única para todos os vértices
        self.simpleShaderProgram.bind()
        self.simpleShaderProgram.useUniformColor(True)
        self.simpleShaderProgram.release()
        
        # ativa o shader programa para permitir configurar uma cor única para todos os vértices
        # e configurar as propriedades da fonte de luz (Sol)
        self.phongShadingProgram.bind()
        self.phongShadingProgram.useUniformMaterialColor(True)
        self.phongShadingProgram.setUniformLightAmbient(np.array([0.2, 0.2, 0.2], dtype=np.float32))
        self.phongShadingProgram.setUniformLightIntensity(np.array([1.0, 1.0, 1.0], dtype=np.float32))
        
        # posiciona a luz na origem (Sol)
        self.lightPosition = glm.vec4(0.0, 0.0, 0.0, 1)
        self.phongShadingProgram.setUniformLightPosition(self.viewMatrix * self.globalTransform * self.lightPosition)
        
        self.phongShadingProgram.setUniformMaterialSpecular(np.array([0.1, 0.1, 0.1]))
        self.phongShadingProgram.setUniformMaterialShininess(5)
        
        #configura a atenuação da luz conforme a distância
        self.phongShadingProgram.useUniformLightAttenuation(True)
        self.phongShadingProgram.setUniformLightConstantAttenuation(1.5)
        self.phongShadingProgram.setUniformLightLinearAttenuation(0.0)
        self.phongShadingProgram.setUniformLightQuadraticAttenuation(0.0)
        self.phongShadingProgram.release()
        
        position_loc = self.phongShadingProgram.getVertexPositionLoc()
        normal_loc = self.phongShadingProgram.getVertexNormalLoc()
        texture_loc = self.phongShadingProgram.getVertexTextureCoordLoc()

        # cria uma malhar de triângulos esférica comum para os objetos esféricos
        sphere_mesh = SphereMesh(1.0, 30, 30)
        sphere_renderer = ModelRenderer(sphere_mesh.getVertexPositions(),
                                        vertex_indices=sphere_mesh.getVertexIndices(),
                                        vertex_normal=sphere_mesh.getVertexNormals(),
                                        vertex_tex=sphere_mesh.getVertexTextureCoord())
        sphere_renderer.setVertexPositionLoc(position_loc)
        sphere_renderer.setVertexNormalLoc(normal_loc)
        sphere_renderer.setVertexTextureCoordLoc(texture_loc)
        
        # laço para cria os objeto onde as informações gráficas dos planetas são armazenadas
        for name in self.planetName:
            self.planets[name] = GraphicObject()
            self.planets[name].model = sphere_mesh
            self.planets[name].renderer = sphere_renderer
            self.planets[name].color = PlanetarySheet[name].color
        
        # cria os objeto para armazenar as informações do ovni
        self.ovni = [GraphicObject()]
        self.ovni.append(GraphicObject())
        self.ovni[0].model = ObjLoader('./cg/models/obj/ovni/Saucer_2(adv).obj')
        self.ovni[1].model = self.ovni[0].model
        for i, item in enumerate(self.ovni[0].model.getItemNames()):
            self.ovni[i].renderer = ModelRenderer(self.ovni[i].model.getVertexPositions(item),
                                                  vertex_normal=self.ovni[i].model.getVertexNormals(item),
                                                  vertex_tex=self.ovni[i].model.getVertexTextureCoord(item))
        self.ovni[1].renderer.setVertexPositionLoc(position_loc)
        self.ovni[1].renderer.setVertexNormalLoc(normal_loc)
        self.ovni[1].renderer.setVertexTextureCoordLoc(texture_loc)
        self.ovni[0].color = np.array([0.0, 1.0, 0.0, 1.0])
        self.ovni[1].color = np.array([0.3, 0.3, 0.3, 1.0])
        
        # cria os objeto para armazenar as informações do meteoro
        self.meteorOrbit = OrbitPolygon(29.1848, 8.1645, 20.8055, 50)
        self.meteor = GraphicObject()
        self.meteor.model = ObjLoader('./cg/models/obj/meteor/OBJ.obj')
        print(self.meteor.model.getItemNames())
        item = self.meteor.model.getItemNames()[0]
        self.meteor.renderer = ModelRenderer(self.meteor.model.getVertexPositions(item),
                                             vertex_normal=self.meteor.model.getVertexNormals(item),
                                             vertex_tex=self.meteor.model.getVertexTextureCoord(item))
        self.meteor.renderer.setVertexPositionLoc(position_loc)
        self.meteor.renderer.setVertexNormalLoc(normal_loc)
        self.meteor.renderer.setVertexTextureCoordLoc(texture_loc)
        self.meteor.color = np.array([0.1, 0.1, 0.1, 1.0])
        
        position_loc = self.simpleShaderProgram.getVertexPositionLoc()
        texture_loc = self.simpleShaderProgram.getVertexTextureCoordLoc()
        
        self.ovni[0].renderer.setVertexPositionLoc(position_loc)
        
        # laço para cria os objetos onde as informações gráficas das órbita são armazenadas
        for name in self.planetName:
            planet_info = PlanetarySheet[name]
            self.orbits[name] = GraphicObject()
            self.orbits[name].model = OrbitPolygon(planet_info.orbitSemiMajorAxis, planet_info.orbitSemiMinorAxis, planet_info.orbitFocus, 50)
            self.orbits[name].renderer = ModelRenderer(self.orbits[name].model.getVertexPositions(), primitive=ModelRenderer.LINE_LOOP)
            self.orbits[name].renderer.setVertexPositionLoc(position_loc)
            self.orbits[name].color = planet_info.color
            self.orbits[name].modelMatrix = glm.rotate(glm.mat4(), glm.radians(planet_info.orbitalInclination), glm.vec3(0.0, 1.0, 0.0))
        
        # cria um objeto para armazenar as informações do quadrado do skybox
        self.square = GraphicObject()
        self.square.model = SquareMesh(skybox_size, skybox_size, 10, 10)
        self.square.renderer = ModelRenderer(self.square.model.getVertexPositions(),
                                             vertex_indices=self.square.model.getVertexIndices(),
                                             vertex_tex=self.square.model.getVertexTextureCoord())
        self.square.renderer.setVertexPositionLoc(position_loc)
        self.square.renderer.setVertexTextureCoordLoc(texture_loc)
        self.square.color = np.array([0.0, 0.0, 0.2, 1.0])
        
        # cria um objeto para armazenar as informações do anel de Saturn
        self.saturnRing = GraphicObject()
        self.saturnRing.model = Torus2DMesh(60, (140000 / AU) * planet_scale_factor, (60900 / AU) * planet_scale_factor)
        self.saturnRing.renderer = ModelRenderer(self.saturnRing.model.getVertexPositions(),
                                                 vertex_tex=self.saturnRing.model.getVertexTextureCoord())
        self.saturnRing.renderer.setVertexPositionLoc(position_loc)
        self.saturnRing.renderer.setVertexTextureCoordLoc(texture_loc)
        self.saturnRing.color = PlanetarySheet['saturn'].color

        # cria um objeto para armazenar as informações do cone do ovni
        self.ovniCone = GraphicObject()
        self.ovniCone.model = ConeMesh(25, PlanetarySheet['jupiter'].radius * 0.2, PlanetarySheet['jupiter'].radius * 0.3)
        self.ovniCone.renderer = ModelRenderer(self.ovniCone.model.getVertexPositions())
        self.ovniCone.renderer.setVertexPositionLoc(position_loc)
        self.ovniCone.color = np.array([0.0, 1.0, 0.0, 0.5], dtype=np.float32)
        
        position_loc = self.sunEffectShaderProgram.getVertexPositionLoc()
        texture_loc = self.sunEffectShaderProgram.getVertexTextureCoordLoc()
        
        # cria um objeto para armazenar as informações gráficas do Sol
        self.sun = GraphicObject()
        self.sun.model = sphere_mesh
        self.sun.renderer = ModelRenderer(sphere_mesh.getVertexPositions(), 
                                          vertex_indices=sphere_mesh.getVertexIndices(),
                                          vertex_tex=sphere_mesh.getVertexTextureCoord())
        self.sun.renderer.setVertexPositionLoc(position_loc)
        self.sun.renderer.setVertexTextureCoordLoc(texture_loc)
        self.sun.color = PlanetarySheet['sun'].color
        self.sun.modelMatrix = glm.scale(glm.mat4(), glm.vec3(PlanetarySheet['sun'].radius, PlanetarySheet['sun'].radius, PlanetarySheet['sun'].radius))
        
        # armazena o momento que o programa começou
        self.startTime = time.time()
    
        # ativa o teste de profundidade e o recurso de culling
        gl.glEnable(gl.GL_DEPTH_TEST);
        gl.glEnable(gl.GL_CULL_FACE)
        
        # configura a cor de background
        gl.glClearColor(1, 1, 1, 1)
        
        # ativa a transparência
        gl.glEnable(gl.GL_BLEND)
        gl.glBlendFunc(gl.GL_SRC_ALPHA, gl.GL_ONE_MINUS_SRC_ALPHA)
        
        # carrega as texturas
        self.loadTextures()
    
    def loadTextures(self):
        
        self.planets['mercury'].texture = Texture('./cg/images/solar_system/2k_mercury.jpg')
        self.planets['venus'].texture = Texture('./cg/images/solar_system/2k_venus_atmosphere.jpg')
        self.planets['earth'].texture = Texture('./cg/images/solar_system/2k_earth_daymap.jpg')
        self.planets['moon'].texture = Texture('./cg/images/solar_system/2k_moon.jpg')
        self.planets['mars'].texture = Texture('./cg/images/solar_system/2k_mars.jpg')
        self.planets['jupiter'].texture = Texture('./cg/images/solar_system/2k_jupiter.jpg')
        self.planets['saturn'].texture = Texture('./cg/images/solar_system/2k_saturn.jpg')
        self.planets['uranus'].texture = Texture('./cg/images/solar_system/2k_uranus.jpg')
        self.planets['neptune'].texture = Texture('./cg/images/solar_system/2k_neptune.jpg')
        self.saturnRing.texture = Texture('./cg/images/solar_system/2k_saturn_ring_alpha.png')
        self.square.texture = Texture('./cg/images/solar_system/8k_stars_milky_way_center.jpg')
        self.ovni[1].texture = Texture('./cg/models/obj/ovni/dark-metal-grid-1.jpg')
        self.meteor.texture = Texture('./cg/models/obj/meteor/Rock_01_Diffuse.jpg')
        
    def paintGL(self):
        
        # limpa o background com a cor especificada
        gl.glClear(gl.GL_COLOR_BUFFER_BIT | gl.GL_DEPTH_BUFFER_BIT)
        
        # calcula o tempo de execução do programa
        self.currentTime = time.time()
        time_elapsed = self.currentTime - self.startTime
        
        # atualiza a posição dos objetos gráficos
        self.updatePositions(time_elapsed)
        
        # renderiza os objetos
        self.renderSkybox()
        self.renderSun(time_elapsed)
        self.renderPlanets()        
        self.renderOrbits()
        self.renderOVNI()
        self.renderObjectWithLighting(self.meteor)
        
        # desabilita o culling para renderizar os dois lados
        gl.glDisable(gl.GL_CULL_FACE)
        self.renderObject(self.saturnRing)
        gl.glEnable(gl.GL_CULL_FACE)
        
        # solicita que o método paintGL seja chamado novamente
        self.update()
    
    def resizeGL(self, width, height):
        
        # atualiza o viewport
        gl.glViewport(0, 0, width, height)
        
        # atualiza a matriz de projeção perspectiva
        aspectRatio = width / height
        self.perspectiveMatrix = glm.perspective(45.0, aspectRatio, 0.1, 200.0)
    
    def updatePositions(self, time_elapsed):
        
        angle = time_elapsed * 2 * math.pi * 0.05

        self.updatePlanetPosition(angle)
        self.updateMoonOrbitPosition(angle)
        self.updateMoonPosition(angle)
        self.updateSaturnRingPosition(angle)
        self.updateOvniPosition(angle)
        self.updateMeteorPosition(angle)
        
    def updatePlanetPosition(self, angle):
        
        # atualiza a transformação de todos os planetas
        for name in self.planetName:

            planet_info = PlanetarySheet[name]
            
            # cria a matriz de translação
            transl_planet_pos = self.createTranslationMatrix(angle, name)
            
            # cria a matriz de rotação da inclinação da órbita
            rot_planet_orbit_incl = glm.rotate(glm.mat4(), glm.radians(planet_info.orbitalInclination), glm.vec3(0.0, 1.0, 0.0))
            
            # cria a matriz de rotação do eixo do planeta
            rot_reorient_axis_planet = glm.rotate(glm.mat4(), glm.radians(90), glm.vec3(1.0, 0.0, 0.0))
            
            # cria uma matriz de escala para deixar o planeta com o tamanho correto
            scale_planet_size = glm.scale(glm.mat4(), glm.vec3(planet_info.radius, planet_info.radius, planet_info.radius))
            
            # calcula a matriz de transformação final do planeta
            self.planets[name].modelMatrix = rot_planet_orbit_incl * transl_planet_pos * scale_planet_size * rot_reorient_axis_planet
        
    def updateMoonOrbitPosition(self, angle):
        
        # cria a matriz de rotação da inclinação da órbita da lua
        rot_moon_orbit_incl = glm.rotate(glm.mat4(), glm.radians(PlanetarySheet['moon'].orbitalInclination), glm.vec3(0.0, 1.0, 0.0))
        
        # cria a matriz de translação
        trans_earth_pos = self.createTranslationMatrix(angle, 'earth')
        
        # cria a matriz de rotação da inclinação da órbita da Terra
        rot_earth_orbit_incl = glm.rotate(glm.mat4(), glm.radians(PlanetarySheet['earth'].orbitalInclination), glm.vec3(0.0, 1.0, 0.0))
        
        # calcula a matriz de transformação final da órbita da lua
        self.orbits['moon'].modelMatrix = rot_earth_orbit_incl * trans_earth_pos * rot_moon_orbit_incl
    
    def updateMoonPosition(self, angle):
        
        # cria a matriz de translação
        transl_moon_pos = self.createTranslationMatrix(angle, 'moon')
        
        # cria a matriz de rotação da inclinação da órbita da lua
        rot_moon_orbit_incl = glm.rotate(glm.mat4(), glm.radians(PlanetarySheet['moon'].orbitalInclination), glm.vec3(0.0, 1.0, 0.0))
        
        # cria a matriz de translação
        transl_earth_pos = self.createTranslationMatrix(angle, 'earth')
        
        # cria a matriz de rotação da inclinação da órbita da Terra
        rot_earth_orbit_incl = glm.rotate(glm.mat4(), glm.radians(PlanetarySheet['earth'].orbitalInclination), glm.vec3(0.0, 1.0, 0.0))
        
        # cria uma matriz de escala para deixar a lua com o tamanho correto
        scale_moon_size = glm.scale(glm.mat4(), glm.vec3(PlanetarySheet['moon'].radius, PlanetarySheet['moon'].radius, PlanetarySheet['moon'].radius))

        # calcula a matriz de transformação final da lua
        self.planets['moon'].modelMatrix = rot_earth_orbit_incl * transl_earth_pos * rot_moon_orbit_incl * transl_moon_pos * scale_moon_size

    def updateSaturnRingPosition(self, angle):      
        
        # cria a matriz de translação
        transl_saturn_pos = self.createTranslationMatrix(angle, 'saturn')
        
        # cria a matriz de rotação da inclinação da órbita de Saturno
        rot_saturn_orbit_incl = glm.rotate(glm.mat4(), glm.radians(PlanetarySheet['saturn'].orbitalInclination), glm.vec3(0.0, 1.0, 0.0)) 
        
        # calcula a matriz de transformação final
        self.saturnRing.modelMatrix = rot_saturn_orbit_incl * transl_saturn_pos 
        
    def updateOvniPosition(self, angle):
        
        # cria a matriz de translação do ovni
        x, y = self.orbits['jupiter'].model.getPositionAt(angle * PlanetarySheet['jupiter'].velocity)
        trans = glm.translate(glm.mat4(), glm.vec3(x, y, PlanetarySheet['jupiter'].radius * 1.4))
        
        # cria a matriz de rotação da inclinação da órbita de Jupiter
        rot_jupiter_orbit_inc = glm.rotate(glm.mat4(), glm.radians(PlanetarySheet['jupiter'].orbitalInclination), glm.vec3(0.0, 1.0, 0.0))
        
        # reorienta o ovni
        rot_reorient = glm.rotate(glm.mat4(), glm.radians(90), glm.vec3(1.0, 0.0, 0.0))
        
        # cria uma matriz de escala para reduzir o tamanho do ovni
        scale = glm.scale(glm.mat4(), glm.vec3(0.07, 0.07, 0.07))
        
        # cria matrix de rotação 
        rot = glm.rotate(glm.mat4(), glm.radians(angle * 40), glm.vec3(0.0, 0.0, 1.0))
        
        # calcula a matriz de transformação final do ovni
        self.ovni[0].modelMatrix = rot_jupiter_orbit_inc * trans * rot * scale * rot_reorient
        self.ovni[1].modelMatrix = rot_jupiter_orbit_inc * trans * rot * scale * rot_reorient
        
        # cria a matriz de translação do cone do ovni
        trans_cone = glm.translate(glm.mat4(), glm.vec3(x, y, PlanetarySheet['jupiter'].radius * 1.0))
        
        # calcula a matriz de transformação final do cone
        self.ovniCone.modelMatrix = rot_jupiter_orbit_inc * trans_cone
    
    def createTranslationMatrix(self, angle, planet_name):
        
        # recupera a posição do planeta na órbita
        # e cria uma matriz de translação para levar o planeta para a nova posição
        x, y = self.orbits[planet_name].model.getPositionAt(angle * PlanetarySheet[planet_name].velocity)
        transl = glm.translate(glm.mat4(), glm.vec3(x, y, 0.0))
        
        return transl
    
    def updateMeteorPosition(self, angle):
        
        # cria a matriz de translação do meteoro
        x, y = self.meteorOrbit.getPositionAt(angle * 0.5)
        trans = glm.translate(glm.mat4(), glm.vec3(x, y, 0.0))
        
        # cria uma matriz de escala para reduzir o tamanho do meteoro
        scale = glm.scale(glm.mat4(), glm.vec3(0.003, 0.003, 0.003))
        
        # cria a matriz de rotação da inclinação da órbita do meteoro
        rot_incl = glm.rotate(glm.mat4(), glm.radians(-10), glm.vec3(0.0, 1.0, 0.0))
        
        # calcula a matriz de transformação final
        self.meteor.modelMatrix = rot_incl * trans * scale
        
    def renderOVNI(self):
        
        self.renderObject(self.ovni[0])
        self.renderObjectWithLighting(self.ovni[1])
        self.renderObject(self.ovniCone)
        
    def renderSkybox(self):
        
        self.renderSquare(glm.radians(0), glm.vec3(0.0, 1.0, 0.0))        
        self.renderSquare(glm.radians(-90), glm.vec3(0.0, 1.0, 0.0))        
        self.renderSquare(glm.radians(90), glm.vec3(0.0, 1.0, 0.0))        
        self.renderSquare(glm.radians(180), glm.vec3(0.0, 1.0, 0.0))
        self.renderSquare(glm.radians(-90), glm.vec3(1.0, 0.0, 0.0))        
        self.renderSquare(glm.radians(90), glm.vec3(1.0, 0.0, 0.0))
        
    def renderSquare(self, rot_angle, rot_axis):
        
        # cria matriz de transformação do quadrado do skybox
        rot = glm.rotate(glm.mat4(), rot_angle, rot_axis)        
        self.square.modelMatrix = glm.translate(rot, glm.vec3(0.0, 0.0, -skybox_size / 2.0))

        self.renderObject(self.square)
        
    def renderSun(self, time_elapsed):
        
        # ativa o shader program que será executado pela GPU
        self.sunEffectShaderProgram.bind()
        
        # atualiza o shader com a transformação dos objetos
        self.sunEffectShaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * self.sun.modelMatrix)
        self.sunEffectShaderProgram.setTime(time_elapsed)
        
        # renderiza o sol
        self.sun.renderer.render()
        
        # desativa o shader program
        self.sunEffectShaderProgram.release()
        
    def renderObjectWithLighting(self, graphic_object):
        
        # ativa o shader program que será executado pela GPU
        self.phongShadingProgram.bind()
        
        # atualiza o shader com a transformação dos objetos
        mv_matrix = self.viewMatrix * self.globalTransform * graphic_object.modelMatrix
        self.phongShadingProgram.setUniformMVPMatrix(self.perspectiveMatrix * mv_matrix)
        self.phongShadingProgram.setUniformModelViewMatrix(mv_matrix)
        self.phongShadingProgram.setUniformMaterialColor(graphic_object.color)
        
        # ativa textura se houver
        if(graphic_object.texture != None):
            self.phongShadingProgram.bindTexture2D(graphic_object.texture.getTextureID())
        
        # renderiza o objeto
        graphic_object.renderer.render()
        
        # desativa a textura
        self.phongShadingProgram.releaseTexture2D()
        
        # desativa o shader program
        self.phongShadingProgram.release()
        
    def renderObject(self, graphic_object):
        
        # ativa o shader program que será executado pela GPU
        self.simpleShaderProgram.bind()
        
        # atualiza o shader com a transformação dos objetos
        self.simpleShaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * self.globalTransform * graphic_object.modelMatrix)
        self.simpleShaderProgram.setUniformColor(graphic_object.color)
        
        # ativa textura se houver
        if(graphic_object.texture != None):
            self.simpleShaderProgram.bindTexture2D(graphic_object.texture.getTextureID())
        
        # renderiza o objeto
        graphic_object.renderer.render()
        
        # desativa a textura
        self.simpleShaderProgram.releaseTexture2D()
        
        # desativa o shader program
        self.simpleShaderProgram.release()
        
    def renderPlanets(self):
        
        # laço para renderizar todos os planetas
        for name in self.planetName:
            self.renderObjectWithLighting(self.planets[name])
        
    def renderOrbits(self):
        
        # laço para renderizar todas as órbitas
        for name in self.planetName:
            self.renderObject(self.orbits[name])      

    # tratamento de evento baseando no algoritmo do link https://learnopengl.com/Getting-started/Camera
    def keyPressEvent(self, event):
        super(MyWidget, self).keyPressEvent(event)
         
        cameraSpeed = 0.5
        new_camera_pos = self.cameraPos
        
        if event.key() == QtCore.Qt.Key_Up:
            new_camera_pos = self.cameraPos + cameraSpeed * self.cameraFront
            
        elif event.key() == QtCore.Qt.Key_Down:
            new_camera_pos = self.cameraPos - cameraSpeed * self.cameraFront
            
        elif event.key() == QtCore.Qt.Key_Left:
            new_camera_pos = self.cameraPos -glm.normalize(glm.cross(self.cameraFront, self.cameraUp)) * cameraSpeed
            
        elif event.key() == QtCore.Qt.Key_Right:
            new_camera_pos = self.cameraPos + glm.normalize(glm.cross(self.cameraFront, self.cameraUp)) * cameraSpeed
            
        if(self.isOutOfLimits(new_camera_pos)):
            return
        
        self.cameraPos = new_camera_pos
        
        # atualiza a matriz de visão
        self.viewMatrix = glm.lookAt(self.cameraPos, self.cameraPos + self.cameraFront, self.cameraUp)
        
        # atualiza a posição da luz (Sol)
        self.phongShadingProgram.bind()
        self.phongShadingProgram.setUniformLightPosition(self.viewMatrix * self.globalTransform * self.lightPosition)
        self.phongShadingProgram.release()

    # testa se a posição da câmera está fora do skybox
    def isOutOfLimits(self, position):
        
        if(position.x < -((skybox_size / 2.0) - 20) or position.x > ((skybox_size / 2.0) - 20)):
            return True
        
        if(position.y < -((skybox_size / 2.0) - 20) or position.y > ((skybox_size / 2.0) - 20)):
            return True
        
        if(position.z < -((skybox_size / 2.0) - 20) or position.z > ((skybox_size / 2.0) - 20)):
            return True
        
        return False
    
    def mousePressEvent(self, event):
        
        self.lastX = event.localPos().x()
        self.lastY = event.localPos().y()
    
    # tratamento de evento baseando no algoritmo do link https://learnopengl.com/Getting-started/Camera
    def mouseMoveEvent(self, event):
               
        xpos = event.localPos().x()
        ypos = event.localPos().y()
        
        xoffset = xpos - self.lastX
        yoffset = self.lastY - ypos
        self.lastX = xpos
        self.lastY = ypos
    
        sensitivity = 0.05
        xoffset *= sensitivity
        yoffset *= sensitivity
    
        self.yaw   -= xoffset
        self.pitch -= yoffset
    
        if(self.pitch > 89.0):
            self.pitch = 89.0
        if(self.pitch < -89.0):
            self.pitch = -89.0
    
        direction = glm.vec3() 
        direction.x = math.cos(glm.radians(self.yaw)) * math.cos(glm.radians(self.pitch))
        direction.y = math.sin(glm.radians(self.pitch))
        direction.z = math.sin(glm.radians(self.yaw)) * math.cos(glm.radians(self.pitch))
        self.cameraFront = glm.normalize(direction)
    
        # atualiza a matriz de visão
        self.viewMatrix = glm.lookAt(self.cameraPos, self.cameraPos + self.cameraFront, self.cameraUp)
        
        # atualiza a posição da luz (Sol)
        self.phongShadingProgram.bind()
        self.phongShadingProgram.setUniformLightPosition(self.viewMatrix * self.globalTransform * self.lightPosition)
        self.phongShadingProgram.release()
        
def main():
    import sys

    app = QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)

    glformat = QtOpenGL.QGLFormat()
    glformat.setVersion(3, 3)
    glformat.setProfile(QtOpenGL.QGLFormat.CoreProfile)
    
    w = MyWidget(glformat)
    w.resize(800, 800)
    w.setWindowTitle('Solar System')
    w.show()
    
    output = app.exec_()
    sys.exit(output)
    
if __name__ == '__main__':
    main()

material_name = Core - vertex_format = N3F_V3F - min = -2.25019907951355 - max = 2.250200033187866
material_name = 

Unimplemented OBJ format statement 'Tf' on line '	Tf 1 1 1'


Hull - vertex_format = T2F_N3F_V3F - min = -5.9427170753479 - max = 6.9427170753479
material_name = wire_225198087 - vertex_format = T2F_N3F_V3F - min = -124.81430053710938 - max = 205.78370666503906
['wire_225198087']
trying to open ./cg/images/solar_system/2k_mercury.jpg
opened file: size= (2048, 1024) format= JPEG mode= RGB
trying to open ./cg/images/solar_system/2k_venus_atmosphere.jpg
opened file: size= (2048, 1024) format= JPEG mode= RGB
trying to open ./cg/images/solar_system/2k_earth_daymap.jpg
opened file: size= (2048, 1024) format= JPEG mode= RGB
trying to open ./cg/images/solar_system/2k_moon.jpg
opened file: size= (2048, 1024) format= JPEG mode= RGB
trying to open ./cg/images/solar_system/2k_mars.jpg
opened file: size= (2048, 1024) format= JPEG mode= RGB
trying to open ./cg/images/solar_system/2k_jupiter.jpg
opened file: size= (2048, 1024) format= JPEG mode= RGB
trying to open ./cg/images/solar_system/2k_saturn.jpg
opened file: size= (2048, 1024) format= JPEG mode= RGB
tryi

SystemExit: 0

C:\Users\Rapha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
